In [1]:
from google.colab import drive
drive.mount('/content/drive')
folder = "/content/drive/MyDrive/CAPSTONE/metadata"

Mounted at /content/drive


In [ ]:
!pip install sentence-transformers matplotlib scipy scikit-learn

In [3]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.9 MB/s eta 0:00:00


In [4]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired

In [5]:
import pandas as pd
import re
exclude = ["Sermons, English.",
           "Sermons,English",
           "Sermons",
           "Early works to 1800.",
           "16th century.",
           "17th century.",
           "-- Sermons -- Early works to 1800."]
exclude = re.compile("|".join(exclude))
def read_data(file):
  sermon_metadata = pd.read_csv(file,header=None)
  pubyears = sermon_metadata[8]
  years = []
  for i, year in enumerate(pubyears):
      if "-" in year:
          year = year.split("-")[0] # earliest possible pub year
          years.append(year)
  subject_headings = sermon_metadata[7]
  titles = list(sermon_metadata[3])
  tcpIDs = sermon_metadata[0]
  info_dict = {}
  for idx, t in enumerate(titles): # place subject headings at the end of titles
      if isinstance(subject_headings[idx],str):
          subjects = subject_headings[idx].split("; ")
          for s, subject in enumerate(subjects):
            subject = re.sub(r"\s+"," ",subject)
            subjects[s] = re.sub(exclude,"",subject)
          title_subjects = f"{t} {'; '.join(subjects)}"
          # title_subjects = re.sub("--"," ",title_subjects)
          # title_subjects = re.sub("\s+"," ",title_subjects)
          info_dict[tcpIDs[idx]] = title_subjects
      else:
          info_dict[tcpIDs[idx]] = t
  return info_dict

In [6]:
docs = read_data(f'{folder}/sermons.csv')
missing_docs = read_data(f'{folder}/sermons_missing.csv')
docs.update(missing_docs)
len(docs)

5859

In [7]:
import json
with open(f'{folder}/corpora.json','r') as file:
        corpora = json.load(file)

era_tcpIDs = {era:{} for era in corpora}
for era in corpora:
    for prefix,ids in corpora[era].items():
        for tcpID in ids:
            era_tcpIDs[era][tcpID] = docs[tcpID]

# Get topic clusters for each era

In [8]:
def get_topics(era):
  representation_model = KeyBERTInspired()
  model = BERTopic(n_gram_range=(1,4),representation_model=representation_model,embedding_model='all-mpnet-base-v2')
  corpus = list(era_tcpIDs[era].values())
  ids = list(era_tcpIDs[era].keys())
  topics, probs = model.fit_transform(corpus)
  # Save topic group for each tcpID
  topics_df = pd.DataFrame({"tcpID": ids, "topic": topics, "probability":probs})
  topics_df.to_csv(f'{folder}/{era}_topics.csv', index=False,header=False)
  # Save topic words for each group
  topic_words = model.get_topics()
  idx_to_topic = []
  for idx, topiclist in topic_words.items():
    idx_to_topic.append({"topic_idx":idx, "topics": "; ".join([t[0] for t in topiclist])})
  topic_words_df = pd.DataFrame(idx_to_topic)
  topic_words_df.to_csv(f"{folder}/{era}_topic_words.csv",header=False,index=False)
  return model

In [13]:
preE = get_topics("pre-Elizabethan")
preE.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,61,-1_sermons of_sacrament_sermon of_sermons,"[sermons of, sacrament, sermon of, sermons, re...",[A sermon had at Paulis by the co[m]mandment o...
1,0,12,0_sermon annexed_sermon annexed vnto it_sermon...,"[sermon annexed, sermon annexed vnto it, sermo...","[The copie of a letter, sent to the ladye Mary..."
2,1,10,1_sermon of_preached before the_he preached be...,"[sermon of, preached before the, he preached b...",[A notable sermon concerninge the ryght vse of...


In [151]:
E = get_topics("Elizabethan")
E.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,16,-1_true martyrs of christ_true martyrs of_mart...,"[true martyrs of christ, true martyrs of, mart...",[The effect of certaine sermons touching the f...
1,0,268,0_sermons of_sermon preached at paules_sermon ...,"[sermons of, sermon preached at paules, sermon...",[A sermon preached in S. Peters Church in Exce...
2,1,18,1_controversial literature catholic church_cat...,"[controversial literature catholic church, cat...","[A copie of a challenge, taken owt [sic] of th..."
3,2,18,2_of the lords supper_the sacrament of the_of ...,"[of the lords supper, the sacrament of the, of...",[A setting open of the subtyle sophistrie of T...


In [149]:
J = get_topics("Jacobean")
J.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,333,-1_in sermon_preached at pauls crosse_in sermo...,"[in sermon, preached at pauls crosse, in sermo...",[The delights of the saints A most comfortable...
1,0,56,0_sermon preached at_sermons preached_fruitful...,"[sermon preached at, sermons preached, fruitfu...","[The way to glory, or, The preaching of the Go..."
2,1,43,1_church controversial literature_the church o...,"[church controversial literature, the church o...","[The height of Israels heathenish idolatrie, i..."
3,2,39,2_sermon preached before the_preached before t...,"[sermon preached before the, preached before t...",[A sermon preached in the cathedrall church of...
4,3,30,3_the epistle of saint_the epistle of_epistle ...,"[the epistle of saint, the epistle of, epistle...",[The triumph of a true Christian described: or...
5,4,30,4_1614 funeral sermons_funeral sermons the_fun...,"[1614 funeral sermons, funeral sermons the, fu...",[Sinnelesse sorrow for the dead a comfortable ...
6,5,26,5_literature sermon against_two sermons_sermon...,"[literature sermon against, two sermons, sermo...",[The yong mans gleanings Gathered out of diuer...
7,6,25,6_in sermon_of sicknesse_two sermons_in sermon...,"[in sermon, of sicknesse, two sermons, in serm...",[A posie of spirituall flowers taken out of th...
8,7,21,7_funerall sermon preached_the funerall of the...,"[funerall sermon preached, the funerall of the...",[A verie godlie and learned sermon treating of...
9,8,18,8_by the bishop of_the bishop of elie_preached...,"[by the bishop of, the bishop of elie, preache...",[A sermon preached before his Maiestie at Whit...


In [155]:
C = get_topics("Carolinian")
C.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,276,-1_in sermon_sermon preached at_sermons_sermon...,"[in sermon, sermon preached at, sermons, sermo...",[The saints qualification: or A treatise I. Of...
1,0,72,0_charles 1625_sermon preached at pauls_1625 1...,"[charles 1625, sermon preached at pauls, 1625 ...",[An exposition of the third chapter of the Epi...
2,1,40,1_sermons on_sermons preached upon_sermons pre...,"[sermons on, sermons preached upon, sermons pr...",[Choice sermons preached upon selected occasio...
3,2,38,2_sermons upon_sermons_severall sermons_sermon,"[sermons upon, sermons, severall sermons, serm...",[Evangelicall sacrifices In xix. sermons. I. T...
4,3,35,3_the funerall of the_funeral sermons the_fune...,"[the funerall of the, funeral sermons the, fun...","[A lasting ievvell, for religious woemen In th..."
5,4,28,4_church of england_of the church_reverend fat...,"[church of england, of the church, reverend fa...","[The Christian divinitie, contained in the div..."
6,5,16,5_of his maiesties chaplaines_maiesties chapla...,"[of his maiesties chaplaines, maiesties chapla...",[A royall edict for military exercises publish...
7,6,15,6_of the lords supper_the lords supper_sacrame...,"[of the lords supper, the lords supper, sacram...",[The saints daily exercise A treatise concerni...
8,7,14,7_fast day sermons_one of the sermons_of the s...,"[fast day sermons, one of the sermons, of the ...",[A sermon preached before the Honourable House...
9,8,13,8_by plague england london_of the plague_plagu...,"[by plague england london, of the plague, plag...","[A sermon intended for Paul's Crosse, but prea..."


In [154]:
CW = get_topics("CivilWar")
CW.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,206,-1_delivered in sermon_sermon preached before ...,"[delivered in sermon, sermon preached before t...",[The Christians hope triumphing in these glori...
1,0,172,0_fast day sermons the_fast day sermons_solemn...,"[fast day sermons the, fast day sermons, solem...",[A firebrand pluckt out of the burning. A serm...
2,1,89,1_war 1642_civil war 1642_war 1642 1649_histor...,"[war 1642, civil war 1642, war 1642 1649, hist...",[Ioseph paralled [sic] by the present Parliame...
3,2,38,2_funeral sermons sermon_funeral sermons the_f...,"[funeral sermons sermon, funeral sermons the, ...","[The life of faith in death, in expectation of..."
4,3,36,3_of the gospel_grace theology_the gospel_theo...,"[of the gospel, grace theology, the gospel, th...",[Practicall divinity: or a helpe through the b...
5,4,30,4_sermon preached at the_sermon preached in_se...,"[sermon preached at the, sermon preached in, s...","[A touch-stone, or, Triall and examination of ..."
6,5,24,5_britain church history_great britain church ...,"[britain church history, great britain church ...","[The sacred and soveraigne church-remedie: or,..."
7,6,21,6_the covenant of grace_the covenant of_covena...,"[the covenant of grace, the covenant of, coven...","[Religious covenanting directed, and covenant-..."
8,7,11,7_delivered in sermon by_delivered in sermon_o...,"[delivered in sermon by, delivered in sermon, ...",[A seasonable sermon for these vnseasonable ti...
9,8,10,8_house of commons bible_of that house bible_i...,"[house of commons bible, of that house bible, ...","[The noble order, or The honour which God conf..."


In [156]:
IR = get_topics("Interregnum")
IR.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,203,-1_of the gospel_in sermon_in sermon preached_...,"[of the gospel, in sermon, in sermon preached,...",[Saul smitten for not smiting Amalek according...
1,0,105,0_funeral sermons sermon_funeral sermons the_i...,"[funeral sermons sermon, funeral sermons the, ...","[The epitaph of a godly man, especially a man ..."
2,1,60,1_sermons preached upon_sermons on_ordination ...,"[sermons preached upon, sermons on, ordination...","[Five sermons, in five several styles; or Waie..."
3,2,47,2_of the gospel_the gospel_sermons preached by...,"[of the gospel, the gospel, sermons preached b...",[A briefe discourse touching a broken heart In...
4,3,46,3_bible corinthians_in sermon_sermon preached ...,"[bible corinthians, in sermon, sermon preached...","[The vanity and mischief of making earthly, to..."
5,4,45,4_1656_1649_1651_sermon preached before the,"[1656, 1649, 1651, sermon preached before the,...",[The true speech delivered on the scaffold by ...
6,5,22,5_the magistrates_preached before the judges_c...,"[the magistrates, preached before the judges, ...",[The judges charge; delivered in a sermon befo...
7,6,17,6_doctrine of justification_justification by f...,"[doctrine of justification, justification by f...",[Confidence dismounted. Or a letter to Mr Rich...
8,7,15,7_great britain church history_of the church_c...,"[great britain church history, of the church, ...","[Provocator provocatus. Or, An answer made to ..."
9,8,14,8_of the lords supper_the lords supper_lords s...,"[of the lords supper, the lords supper, lords ...",[Two treatises: I. The saints communion with J...


In [157]:
CII = get_topics("CharlesII")
CII.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,547,-1_in sermon preached_sermon preached at_in se...,"[in sermon preached, sermon preached at, in se...",[Ebdomas embolimaios a supplement to the eniau...
1,0,210,0_funeral sermons sermon_funeral sermons sermo...,"[funeral sermons sermon, funeral sermons sermo...",[The life and death of the godly man exemplifi...
2,1,144,1_church of england bible_of england bible_pre...,"[church of england bible, of england bible, pr...",[A sermon preached before the Right Honourable...
3,2,72,2_dissenters religious england_church controve...,"[dissenters religious england, church controve...",[The tryal of spirits both in teachers & heare...
4,3,55,3_some sermons_of the gospel_of the gospel at_...,"[some sermons, of the gospel, of the gospel at...","[Great and precious promises; or, Some sermons..."
5,4,52,4_1660 1688_sermon preached before the_preache...,"[1660 1688, sermon preached before the, preach...",[Mercy in the midst of judgment by a gracious ...
6,5,51,5_fast day sermons sermon_fast day sermons_the...,"[fast day sermons sermon, fast day sermons, th...","[A sermon preached on the fast-day, November 1..."
7,6,46,6_september 1683 being_september 1683_gunpowde...,"[september 1683 being, september 1683, gunpowd...",[A sermon preach'd before the King in the Cath...
8,7,42,7_of england sermons_church of england sermons...,"[of england sermons, church of england sermons...",[Fifty sermons preached at the parish-church o...
9,8,41,8_history charles ii 1660_charles ii 1660 1685...,"[history charles ii 1660, charles ii 1660 1685...","[God save the King: or, A sermon preach'd at L..."


In [158]:
JII = get_topics("JamesII")
JII.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,8,-1_fast day sermons_the year 1666_the year 166...,"[fast day sermons, the year 1666, the year 166...","[Londons remembrancer, or, A sermon preached a..."
1,0,235,0_sermon preached before the_sermon preached a...,"[sermon preached before the, sermon preached a...",[A vindication of a passage in Dr. Sherlock's ...
2,1,21,1_rebellion 1685_of july 1685_26th of july 168...,"[rebellion 1685, of july 1685, 26th of july 16...",[The character of a rebel a sermon preached at...


In [159]:
WM = get_topics("WilliamAndMary")
WM.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,422,-1_sermon preached before the_preached before ...,"[sermon preached before the, preached before t...",[A sermon preached before the Right Honourable...
1,0,155,0_1691 funeral sermons_funeral sermons sermon_...,"[1691 funeral sermons, funeral sermons sermon,...",[A funeral-sermon upon occasion of the death o...
2,1,117,1_of england 1650 1702_of england 1650_england...,"[of england 1650 1702, of england 1650, englan...",[A sermon preach'd before the Honourable House...
3,2,53,2_of england sermons_church of england sermons...,"[of england sermons, church of england sermons...",[Twelve sermons preached on several occasions....
4,3,42,3_fast day sermons sermon_fast day sermons_the...,"[fast day sermons sermon, fast day sermons, th...",[A sermon preach'd before the honourable House...
5,4,41,4_the reformation of manners_reformation of ma...,"[the reformation of manners, reformation of ma...",[A sermon preach'd to the societies for reform...
6,5,32,5_1662 1694 funeral sermons_1694 funeral sermo...,"[1662 1694 funeral sermons, 1694 funeral sermo...",[A sermon preach'd at the chappel royal in the...
7,6,28,6_of england bible_church of england bible_bis...,"[of england bible, church of england bible, bi...",[A sermon preached before the King & Queen at ...
8,7,26,7_charity sermon_charity given to_of charity_c...,"[charity sermon, charity given to, of charity,...",[Publick charity a sermon preached before the ...
9,8,24,8_scripture justification_justification christ...,"[scripture justification, justification christ...",[Christ's righteousness a believer's surest pl...


# Stratified Sampling

In [4]:
import pandas as pd
import json
with open(f'{folder}/corpora.json','r') as file:
        corpora = json.load(file)

In [13]:
era_counts = {era:0 for era in corpora}
for era in corpora:
    for prefix,ids in corpora[era].items():
          era_counts[era] += len(ids)
total = sum(era_counts.values())
for era, count in era_counts.items():
    era_counts[era] = round((count / total) * 300)
era_counts

{'pre-Elizabethan': 4,
 'Elizabethan': 16,
 'Jacobean': 35,
 'Carolinian': 28,
 'CivilWar': 33,
 'Interregnum': 29,
 'CharlesII': 78,
 'JamesII': 14,
 'WilliamAndMary': 63}

In [36]:
samples = {}
for era in era_counts:
  data = pd.read_csv(f"{folder}/{era}_topics.csv",header=None,names=['tcpID','group'],usecols=[0,1])
  num_groups = len(data['group'].unique())
  total = 22 # era_counts[era]
  sample = data.groupby('group', group_keys=False).apply(lambda x: x.sample(round(total/num_groups)))
  samples[era] = list(sorted(sample['tcpID']))
  print(era)
  print(sample)

pre-Elizabethan
     tcpID  group
2   B00958     -1
35  A06329     -1
79  A68325     -1
61  A19734     -1
26  A02882     -1
14  A07584     -1
56  A17636     -1
8   A07230      0
42  A04512      0
5   A06508      0
47  A04511      0
6   A00609      0
46  A08050      0
19  A02883      0
16  A09915      1
27  A05394      1
10  A07260      1
28  A05142      1
18  A05141      1
59  A19143      1
22  A05158      1
Elizabethan
      tcpID  group
254  A12386     -1
313  A72347     -1
248  A16144     -1
171  A14350     -1
310  A73748     -1
1    B15274     -1
228  A17318      0
169  A11247      0
288  A69056      0
136  A17722      0
45   A05404      0
103  A04416      0
12   B07515      1
121  A09101      1
133  A07105      1
233  A19272      1
304  A68078      1
55   A08197      1
145  A10233      2
294  A68376      2
204  A12345      2
305  A68172      2
148  A17190      2
79   A03909      2
Jacobean
      tcpID  group
297  A01700     -1
57   A03419      0
169  A03335      1
638  A22051     

<ipython-input-36-aca3edf5db79>:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample = data.groupby('group', group_keys=False).apply(lambda x: x.sample(round(total/num_groups)))
<ipython-input-36-aca3edf5db79>:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample = data.groupby('group', group_keys=False).apply(lambda x: x.sample(round(total/num_groups)))
<ipython-input-36-aca3edf5db79>:6: DeprecationWa

In [37]:
with open(f"{folder}/samples.json","w+") as f:
    json.dump(samples,f)